# ESAA 팀 과제 
# 2019년 10월 11일 
## [3번 주제] like 와 dislike (호감/비호감) 비율 구한 뒤, 어떤 카테고리인지 파악하기

In [130]:
# 패키지 불러오기
from pandas import *
import pandas as pd

In [131]:
# 현재 위치 확인 
import os 
os.getcwd() # 현재 디렉토리 위치 확인 

'C:\\Users\\Chanmi Yoo\\Desktop\\ESAA\\191011 ESAA 팀'

In [146]:
# 데이터 불러오기 
input_fd = open('KRvideos.csv',errors = 'backslashreplace',encoding='utf-8')
KRvideos = pd.read_csv(input_fd)

print(KRvideos.head())

      video_id trending_date                       title channel_title  \
0  RxGQe4EeEpA      17.14.11          좋아 by 민서_윤종신_좋니 답가        라푸마코리아   
1  hH7wVE8OlQ0      17.14.11            JSA 귀순 북한군 총격 부상        Edward   
2  9V8bnWUmE9U      17.14.11  나몰라패밀리 운동화 영상 2탄 (빼빼로데이버젼)     나몰라패밀리 핫쇼   
3  0_8py-t5R80      17.14.11  이명박 출국 현장, 놓치면 안되는 장면         미디어몽구   
4  bk55RbxiQdI      17.14.11     김장겸은 물러갔다 MBC 노조 환호와 눈물        NocutV   

   category_id              publish_time  \
0           22  2017-11-13T07:07:36.000Z   
1           25  2017-11-13T10:59:16.000Z   
2           22  2017-11-11T07:16:08.000Z   
3           25  2017-11-12T11:19:52.000Z   
4           25  2017-11-13T11:08:59.000Z   

                                                tags   views  likes  dislikes  \
0                      라푸마|"윤종신"|"좋니"|"좋아"|"샬레"|"민서"  156130   1422        40   
1        JSA|"귀순"|"북한군"|"총격"|"부상"|"JSA 귀순 북한군 총격 부상"   76533    211        28   
2                아디다스|"빼빼로"|"핫쇼"|"나몰라패밀리"|"대학로"

In [134]:
# 카데고리 설명 json 파일 불러오기 
import json
from pprint import pprint

with open('KR_category_id.json') as data_file:    
    data = json.load(data_file)
    
pprint(data)

{'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/1v2mrzYSYG6onNLt2qTj13hkQZk"',
 'items': [{'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
            'id': '1',
            'kind': 'youtube#videoCategory',
            'snippet': {'assignable': True,
                        'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                        'title': 'Film & Animation'}},
           {'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"',
            'id': '2',
            'kind': 'youtube#videoCategory',
            'snippet': {'assignable': True,
                        'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                        'title': 'Autos & Vehicles'}},
           {'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
            'id': '10',
            'kind': 'youtube#videoCategory',
            'snippet': {'assignable': True,
                        'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                        'title': 'Music'}},
 

In [143]:
# 카테고리 설명 데이터프레임 만들기 
ID = []
title = []

for i in range(len(data["items"])):
    ID.append(data["items"][i]["id"])
    title.append(data["items"][i]["snippet"]["title"])

category_desc = pd.DataFrame({'category_id':ID, 
                             'category': title})
print(category_desc.head())

  category_id          category
0           1  Film & Animation
1           2  Autos & Vehicles
2          10             Music
3          15    Pets & Animals
4          17            Sports


In [281]:
# 문자열 변환 
category_desc["category_id"] = category_desc["category_id"].apply(str)
KRvideos["category_id"] = KRvideos["category_id"].apply(str)

# 기존 KR 데이터프레임과 카테고리 설명 데이터프레임을 합치고, KR에 저장하기 
KR = pd.merge(KRvideos, category_desc, on="category_id", how='left')
print(KR.head(2))

      video_id trending_date               title channel_title category_id  \
0  RxGQe4EeEpA      17.14.11  좋아 by 민서_윤종신_좋니 답가        라푸마코리아          22   
1  hH7wVE8OlQ0      17.14.11    JSA 귀순 북한군 총격 부상        Edward          25   

               publish_time                                         tags  \
0  2017-11-13T07:07:36.000Z                라푸마|"윤종신"|"좋니"|"좋아"|"샬레"|"민서"   
1  2017-11-13T10:59:16.000Z  JSA|"귀순"|"북한군"|"총격"|"부상"|"JSA 귀순 북한군 총격 부상"   

    views  likes  dislikes  comment_count  \
0  156130   1422        40            272   
1   76533    211        28            113   

                                   thumbnail_link  comments_disabled  \
0  https://i.ytimg.com/vi/RxGQe4EeEpA/default.jpg              False   
1  https://i.ytimg.com/vi/hH7wVE8OlQ0/default.jpg              False   

   ratings_disabled  video_error_or_removed  \
0             False                   False   
1             False                   False   

                                         

### [좋아요(likes) 상위 25%의 카테고리 분포]

In [193]:
# 좋아요 내림차순으로 정렬 후 KR에 저장 
KR_likes = KR.sort_values(by=['likes'], axis=0, ascending=False)

# 좋아요 상위 25% 추출 후 KR_25에 저장 
KR_likes25 = KR.iloc[1:round(len(KR_likes)*0.25)]

In [194]:
# 좋아요 상위 25%의 카테고리 분포 살펴보기 
KR_likes25['category'].astype(object)
print(KR_likes25.category.value_counts())

Entertainment           2377
Music                   1153
People & Blogs          1130
Comedy                  1031
News & Politics          976
Film & Animation         569
Gaming                   478
Pets & Animals           395
Howto & Style            200
Sports                   175
Education                 84
Science & Technology      47
Travel & Events           15
Autos & Vehicles           6
Name: category, dtype: int64


#### 좋아요 수가 많은(상위 25% 수준) 유투브 동영상의 카데고리 분포를 살펴본 결과, Entertainment가 2377개로 가장 많았고, Music과 People&Blogs가 각각 1153개와 1130개로 그 뒤를 이었으며, Comedy도 1031개로 많았다. 

In [195]:
print(round(KR_likes25.category.value_counts()/len(KR_likes25)*100, 2))

Entertainment           27.51
Music                   13.34
People & Blogs          13.08
Comedy                  11.93
News & Politics         11.29
Film & Animation         6.58
Gaming                   5.53
Pets & Animals           4.57
Howto & Style            2.31
Sports                   2.03
Education                0.97
Science & Technology     0.54
Travel & Events          0.17
Autos & Vehicles         0.07
Name: category, dtype: float64


#### Entertainment 27.51%, Music 13.34%, People & Blogs 13.08%, Comedy 11.93%, News & Politics 11.29%로 집계되었다.

#### 즉, 한국인에게 좋아요를 많이 받은 유투브 영상은 주로 연예, 음악, v-log 콘텐츠인 것으로 드러났으며, 교육, 과학기술, 여행, 자동차 관련 콘텐츠는 좋아요 수가 높지 않았다.

### [싫어요(dislikes) 상위 25%의 카테고리 분포]

In [186]:
# 같은 방식으로 싫어요 상위 25%의 카테고리 분포를 살펴보자

KR_dislikes = KR.sort_values(by=['dislikes'], axis=0, ascending=False)
KR_dislikes25 = KR.iloc[1:round(len(KR_dislikes)*0.25)]
KR_dislikes25['category'].astype(object)
print(KR_dislikes25.category.value_counts())

Entertainment           2377
Music                   1153
People & Blogs          1130
Comedy                  1031
News & Politics          976
Film & Animation         569
Gaming                   478
Pets & Animals           395
Howto & Style            200
Sports                   175
Education                 84
Science & Technology      47
Travel & Events           15
Autos & Vehicles           6
Name: category, dtype: int64


#### 싫어요 수가 많은 카테고리는 Entertainment(2377개), Music(1153개), People & Blogs(1130개), Comedy(1031개) 순인 것으로 나타났다. 


In [190]:
print(round(KR_dislikes25.category.value_counts()/len(KR_dislikes25)*100, 2))

Entertainment           27.51
Music                   13.34
People & Blogs          13.08
Comedy                  11.93
News & Politics         11.29
Film & Animation         6.58
Gaming                   5.53
Pets & Animals           4.57
Howto & Style            2.31
Sports                   2.03
Education                0.97
Science & Technology     0.54
Travel & Events          0.17
Autos & Vehicles         0.07
Name: category, dtype: float64


#### 비율을 살펴보면, Entertainment가 27.51%로 가장 많았고, Music(13.34%)과 People & Blogs(13.08%)로 그 뒤를 이었으며, Comedy(11.93%)와 News & Politics(11.29%)도 싫어요를 많이 받은 편이었다.

In [285]:
print(KR_dislikes25["title"].head(20))

30583                  BTS (방탄소년단) 'FAKE LOVE' Official MV
30352                  BTS (방탄소년단) 'FAKE LOVE' Official MV
30486                  BTS (방탄소년단) 'FAKE LOVE' Official MV
30291                  BTS (방탄소년단) 'FAKE LOVE' Official MV
30171                  BTS (방탄소년단) 'FAKE LOVE' Official MV
30069                  BTS (방탄소년단) 'FAKE LOVE' Official MV
29969                  BTS (방탄소년단) 'FAKE LOVE' Official MV
4856     YouTube Rewind: The Shape of 2017 | #YouTubeRe...
4678     YouTube Rewind: The Shape of 2017 | #YouTubeRe...
2788     BTS (방탄소년단) 'MIC Drop (Steve Aoki Remix)' Offi...
4018     Marvel Studios' Avengers: Infinity War Officia...
3750     Marvel Studios' Avengers: Infinity War Officia...
2563     BTS (방탄소년단) 'MIC Drop (Steve Aoki Remix)' Offi...
28605    BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity'...
3544     Marvel Studios' Avengers: Infinity War Officia...
28371    BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity'...
28554    Childish Gambino - This Is America (Official V.

In [286]:
print(KR_likes25["title"].head(20))

30583                  BTS (방탄소년단) 'FAKE LOVE' Official MV
30352                  BTS (방탄소년단) 'FAKE LOVE' Official MV
30486                  BTS (방탄소년단) 'FAKE LOVE' Official MV
30291                  BTS (방탄소년단) 'FAKE LOVE' Official MV
30171                  BTS (방탄소년단) 'FAKE LOVE' Official MV
30069                  BTS (방탄소년단) 'FAKE LOVE' Official MV
29969                  BTS (방탄소년단) 'FAKE LOVE' Official MV
4856     YouTube Rewind: The Shape of 2017 | #YouTubeRe...
4678     YouTube Rewind: The Shape of 2017 | #YouTubeRe...
2788     BTS (방탄소년단) 'MIC Drop (Steve Aoki Remix)' Offi...
4018     Marvel Studios' Avengers: Infinity War Officia...
3750     Marvel Studios' Avengers: Infinity War Officia...
2563     BTS (방탄소년단) 'MIC Drop (Steve Aoki Remix)' Offi...
28605    BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity'...
3544     Marvel Studios' Avengers: Infinity War Officia...
28371    BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity'...
28554    Childish Gambino - This Is America (Official V.

In [278]:
total_25 = pd.concat([KR_likes25, KR_dislikes25])
total_25["duplicated"] = total_25.duplicated()

# 좋아요 상위 25% 중에서 싫어요 상위 25%와 겹치지 않는 영상들  
unique_25 = total_25.loc[total_25["duplicated"] == False, :]

# 좋아요 상위 25% 영상 개수
print(KR_likes25.shape[0])
# 싫어요 상위 25% 영상 개수
print(KR_dislikes25.shape[0])
# 좋아요/싫어요 둘 중 하나에만 해당되는 영상 개수 
print(unique_25.shape[0])

8641
8641
7873


#### 상위 25%에 해당하는 영상의 개수는 8641개인데, 그 중 7873개의 영상이 겹치지 않는다. 즉 좋아요와 싫어요, 모두를 많이 받은 영상은 768개로 약 9%를 차지한다.  따라서 겹치는 영상이 많기 때문에 좋아요 카테고리와 싫어요 카테고리의 순위가 일치한다고 볼 수 없다. 
#### 단, 좋아요 최상위권과 싫어요 최상위권은 다소 중복되는 경향이 있었는데, 제목을 살펴보면 대부분 BTS(방탄소년단)의 공식 뮤직비디오였다. 이러한 현상은 팬덤 간 상호 견제, BTS 유행에 따른 호기심과 대중의 적극적 평가 등이 영향을 미쳤을 것으로 예상된다. 

## [결론]

### 한국 Youtube 동영상 중에서 가장 좋아요 혹은 싫어요가 많은 카테고리는 Entertainment였으며, 좋아요가 많은 카테고리 순위와 싫어요가 많은 카테고리 순위가 동일했다. 
### 본 분석을 통해 Entertainment, Music, People&Blogs, Comedy 콘텐츠에 대한 한국인의 반응이 활발하게 일어난다는 것을 알 수 있었다. 